In [1]:
from langchain_groq import ChatGroq

In [15]:


llm = ChatGroq(
    temperature=0, 
    groq_api_key='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', 
    model_name="llama-3.3-70b-versatile"
)

response = llm.invoke("The first person to land on moon was ...")
print(response.content)

The first person to land on the moon was Neil Armstrong. He stepped out of the lunar module Eagle and onto the moon's surface on July 20, 1969, during the Apollo 11 mission. Armstrong famously declared, "That's one small step for man, one giant leap for mankind," as he became the first human to set foot on the moon.


In [16]:
import os
groq_api_key = os.getenv('GROQ_API_KEY')


In [17]:


from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://www.amazon.jobs/en/jobs/2895878/ml-quality-process-lead-portuguese-customer-engagement-technology")

page_data = loader.load().pop().page_content
print(page_data)


ML Quality Process Lead - Portuguese, Customer Engagement Technology - Job ID: 2895878 | Amazon.jobs
Skip to main contentHomeTeamsLocationsJob categoriesMy careerMy applicationsMy profileAccount securitySettingsSign outResourcesDisability accommodationsBenefitsInclusive experiencesInterview tipsLeadership principlesWorking at AmazonFAQ×ML Quality Process Lead - Portuguese, Customer Engagement TechnologyJob ID: 2895878 | Amazon.com Services LLCApply nowDESCRIPTIONThe Customer Engagement Technology (CET) organization powers customer service by developing elegant customer and CS Associate (CSA) facing products globally. These products offer effortless self-service and automation solutions to our customers. If customers prefer to interact with a human, we enable CSAs to effectively and elegantly solve customers’ issues using our associate-facing products powered through human-centered design.We are seeking a ML (Machine Learning) Quality Process Lead, who are fluent in Portuguese and Engli

In [18]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
        """
        ### SCRAPED TEXT FROM WEBSITE:
        {page_data}
        ### INSTRUCTION:
        The scraped text is from the career's page of a website.
        Your job is to extract the job postings and return them in JSON format containing the 
        following keys: `role`, `experience`, `skills` and `description`.
        Only return the valid JSON.
        ### VALID JSON (NO PREAMBLE):    
        """
)

chain_extract = prompt_extract | llm 
res = chain_extract.invoke(input={'page_data':page_data})
type(res.content)

str

In [19]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
json_res

{'role': 'ML Quality Process Lead - Portuguese, Customer Engagement Technology',
 'experience': 'Experience in creating and managing ML annotation processes, testing models, and quality assurance methodologies',
 'skills': ['Language fluency in Portuguese (Native-level) and English',
  'Analytical and problem-solving skills',
  'Ability to thoroughly investigate and identify misalignment between annotations and SOPs',
  'Strong ownership and accountability to meet SLAs, and proactive communication regarding blockers, and proposed solutions',
  'Ability to collaborate closely with cross-functional teams'],
 'description': 'The Customer Engagement Technology (CET) organization powers customer service by developing elegant customer and CS Associate (CSA) facing products globally. We are seeking a ML (Machine Learning) Quality Process Lead, who are fluent in Portuguese and English, to join the Omni Machine Learning Data Associate (MLDA) team within CET to help manage quality management pro

In [20]:
type(json_res)

dict

In [21]:
import pandas as pd

df = pd.read_csv("my_portfolio.csv")
df

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
5,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
6,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
7,"Magento, PHP, MySQL",https://example.com/magento-portfolio
8,"React Native, Node.js, MongoDB",https://example.com/react-native-portfolio
9,"iOS, Swift, Core Data",https://example.com/ios-portfolio


In [22]:
import uuid
import chromadb

client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name="portfolio")

if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents=row["Techstack"],
                       metadatas={"links": row["Links"]},
                       ids=[str(uuid.uuid4())])

In [23]:
links = collection.query(query_texts=["experience in python","experience in java"], n_results=2).get('metadatas', [])
links


[[{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/python-portfolio'}],
 [{'links': 'https://example.com/java-portfolio'},
  {'links': 'https://example.com/android-portfolio'}]]

In [24]:
links = collection.query(query_texts=job['skills'], n_results=2).get('metadatas', [])
links

[[{'links': 'https://example.com/java-portfolio'},
  {'links': 'https://example.com/ios-ar-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/angular-portfolio'}],
 [{'links': 'https://example.com/java-portfolio'},
  {'links': 'https://example.com/ml-python-portfolio'}],
 [{'links': 'https://example.com/full-stack-js-portfolio'},
  {'links': 'https://example.com/typescript-frontend-portfolio'}],
 [{'links': 'https://example.com/xamarin-portfolio'},
  {'links': 'https://example.com/devops-portfolio'}]]

In [25]:
job 

{'role': 'ML Quality Process Lead - Portuguese, Customer Engagement Technology',
 'experience': 'Experience in creating and managing ML annotation processes, testing models, and quality assurance methodologies',
 'skills': ['Language fluency in Portuguese (Native-level) and English',
  'Analytical and problem-solving skills',
  'Ability to thoroughly investigate and identify misalignment between annotations and SOPs',
  'Strong ownership and accountability to meet SLAs, and proactive communication regarding blockers, and proposed solutions',
  'Ability to collaborate closely with cross-functional teams'],
 'description': 'The Customer Engagement Technology (CET) organization powers customer service by developing elegant customer and CS Associate (CSA) facing products globally. We are seeking a ML (Machine Learning) Quality Process Lead, who are fluent in Portuguese and English, to join the Omni Machine Learning Data Associate (MLDA) team within CET to help manage quality management pro

In [26]:
job = json_res
job['skills']

['Language fluency in Portuguese (Native-level) and English',
 'Analytical and problem-solving skills',
 'Ability to thoroughly investigate and identify misalignment between annotations and SOPs',
 'Strong ownership and accountability to meet SLAs, and proactive communication regarding blockers, and proposed solutions',
 'Ability to collaborate closely with cross-functional teams']

In [27]:
prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}
        
        ### INSTRUCTION:
        You are Mohan, a business development executive at AtliQ. AtliQ is an AI & Software Consulting company dedicated to facilitating
        the seamless integration of business processes through automated tools. 
        Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability, 
        process optimization, cost reduction, and heightened overall efficiency. 
        Your job is to write a cold email to the client regarding the job mentioned above describing the capability of AtliQ 
        in fulfilling their needs.
        Also add the most relevant ones from the following links to showcase Atliq's portfolio: {link_list}
        Remember you are Mohan, BDE at AtliQ. 
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):
        
        """
        )

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)

Subject: Expert ML Quality Process Solutions for Customer Engagement Technology

Dear Hiring Manager,

I came across the job posting for an ML Quality Process Lead - Portuguese, Customer Engagement Technology, and I am excited to introduce AtliQ, an AI & Software Consulting company that can fulfill your requirements. With our expertise in machine learning and software development, we can help you manage quality management processes to analyze, improve annotation, testing, and contact reading accuracy for your Customer Service Large Language Models (LLMs).

At AtliQ, we have a proven track record of delivering tailored solutions that foster scalability, process optimization, cost reduction, and heightened overall efficiency. Our team of experts has experience in creating and managing ML annotation processes, testing models, and quality assurance methodologies. We are confident that our capabilities align with your needs, and we would be delighted to collaborate with your team.

Our port